In [1]:
# Подключаем библиотеки

import pandas as pd
import numpy as np
import json
import csv
import os
import re
import datetime as dt

In [2]:
# Получаем список дирректорий с CSV файлами

dirs = os.listdir(path="D:\\Data\\Kinopoisk\\CSV\\characteristics")

In [3]:
# Объединяем все CSV файлы из нескольких дирректорий

lnk = 'D:\\Data\\Kinopoisk\\CSV\\characteristics\\{}'
dataset = pd.DataFrame()
for i in dirs:
    lst = lnk.format(i)
    fls = os.listdir(path=lst)
    for j in fls:
        d = pd.read_csv(lst+'\\'+j)
        dataset = pd.concat([dataset, d], ignore_index=True)

In [4]:
# Обзор итогового датасета

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345500 entries, 0 to 345499
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  345500 non-null  int64 
 1   name        345500 non-null  object
 2   links       345500 non-null  object
 3   year        345500 non-null  int64 
 4   type        345500 non-null  object
 5   char        345500 non-null  object
dtypes: int64(2), object(4)
memory usage: 15.8+ MB


In [5]:
# Создаем копию датасета

df = dataset.copy()
df.sample(10)

,Unnamed: 0,name,links,year,type,char
197916,197916,Alien Territory,https://www.kinopoisk.ru/film/1250770/,2019,film,"[[{'name': 'Alien Territory (2019)', 'descript..."
147235,147235,Dirty,https://www.kinopoisk.ru/film/1311043/,2018,film,"[[{'name': 'Dirty (2018)', 'description': 'non..."
106164,106164,The Bench,https://www.kinopoisk.ru/film/4440784/,2017,film,"[[{'name': 'The Bench (2017)', 'description': ..."
34814,39314,Держись!,https://www.kinopoisk.ru/film/979986/,2016,film,"[[{'name': 'Держись! (2016)', 'description': '..."
313649,314149,Al Límite,https://www.kinopoisk.ru/series/5115127/,2021,series,"[[{'name': 'Al Límite (сериал 2021)', 'descrip..."
15236,21736,The Last Picture,https://www.kinopoisk.ru/film/934776/,2015,film,"[[{'name': 'The Last Picture (2015)', 'descrip..."
246492,246492,A Dead Bastard's Voice,https://www.kinopoisk.ru/film/1432883/,2020,film,"[[{'name': ""A Dead Bastard's Voice (2020)"", 'd..."
243441,243441,A Chapter In Life,https://www.kinopoisk.ru/film/1277313/,2020,film,"[[{'name': 'A Chapter In Life (2020)', 'descri..."
7122,14622,Daytona,https://www.kinopoisk.ru/film/851213/,2015,film,"[[{'name': 'Daytona (2015)', 'description': 'n..."
90625,90125,The Ways We Fail: a recollection,https://www.kinopoisk.ru/film/1447342/,2017,film,[[{'name': 'The Ways We Fail: a recollection (...


In [6]:
# Убираем строки с пустыми списками характеристик (выдавали ошибку 404)

df = df[df['char'] != '[]']

In [7]:
# Преобразуем стрковый формат характеристик в список

df['charlist'] = df['char'].apply(lambda x: eval(x))

In [8]:
# Сохраняем целевые поля в датасете

df1 = df[['name', 'links', 'year', 'type', 'charlist']].copy()
df1.sample(8)

,name,links,year,type,charlist
150816,Tropas especiales de agitación,https://www.kinopoisk.ru/film/1420964/,2018,film,[[{'name': 'Tropas especiales de agitación (20...
274553,Jayne & Elvis,https://www.kinopoisk.ru/film/4897541/,2020,film,"[[{'name': 'Jayne & Elvis (2020)', 'descriptio..."
176002,Jump,https://www.kinopoisk.ru/film/1226691/,2018,film,"[[{'name': 'Jump (2018)', 'description': 'none..."
58001,Glass Slippers,https://www.kinopoisk.ru/film/5100280/,2016,film,"[[{'name': 'Glass Slippers (2016)', 'descripti..."
158784,My Son Hates My Boyfriend,https://www.kinopoisk.ru/film/1158309/,2018,film,"[[{'name': 'My Son Hates My Boyfriend (2018)',..."
88716,Curated Illusions,https://www.kinopoisk.ru/film/1073174/,2017,film,"[[{'name': 'Curated Illusions (2017)', 'descri..."
64867,That Guy,https://www.kinopoisk.ru/film/978988/,2016,film,"[[{'name': 'That Guy (2016)', 'description': '..."
329715,The Shop Web Series,https://www.kinopoisk.ru/series/5200442/,2022,series,[[{'name': 'The Shop Web Series (сериал 2022)'...


In [9]:
# Разбиваем словарь с названием, годом выпуска и описанием фильма по столбцам

df1['Полное название'] = df1['charlist'].apply(lambda x: x[0][0]['name'])
df1['Рейтинг'] = df1['charlist'].apply(lambda x: x[0][0]['rating'])
df1['Оценок'] = df1['charlist'].apply(lambda x: x[0][0]['grades'])
df1['Полное описание'] = df1['charlist'].apply(lambda x: x[0][0])

In [10]:
# Определяем функцию для составления списка всех уникальных заголовков характеристик

merg = []

def keys_unique(row):
    dcts = row[1]
    for i in dcts:
        k = list(i)
        if k not in merg:
            merg.append(k)


In [11]:
# Применяем функцию к серии со списком характеристик

df1['charlist'].apply(keys_unique)

0         None
1         None
2         None
3         None
4         None
          ... 
345495    None
345496    None
345497    None
345498    None
345499    None
Name: charlist, Length: 344814, dtype: object

In [12]:
# Список заголовков всех характеристик фильма

merg

[['Год производства'],
 ['Платформа'],
 ['Страна'],
 ['Жанр'],
 ['Слоган'],
 ['Режиссер'],
 ['Сценарий'],
 ['Продюсер'],
 ['Оператор'],
 ['Композитор'],
 ['Художник'],
 ['Монтаж'],
 ['Премьера в Росcии'],
 ['Премьера в мире'],
 ['Цифровой релиз'],
 ['Релиз на DVD'],
 ['Релиз на Blu-ray'],
 ['Возраст'],
 ['Время'],
 ['Бюджет'],
 ['Сборы в США'],
 ['Сборы в мире'],
 ['Сборы в России'],
 ['Рейтинг MPAA'],
 ['Зрители'],
 ['Ре-релиз (РФ)'],
 ['Директор фильма'],
 ['Маркетинг']]

In [13]:
# Определяем функцию распределения значений характеристик по отдельным столбцам

def podbor(row):
    global i
    x = row[1]
    for j in x:
        if i == list(j):
            n = j.get(i[0])
            break
        else:
            n = 'none'
    return n   

In [14]:
# Выделяем каждую характеристику из общего списка в отдельный столбец

for i in merg:
    df1[i[0]] = df1['charlist'].apply(podbor)

In [15]:
# Определяем функцию по выделению значения характеристики из списка

def sort(row):
    if type(row) == list:
        lst = []
        for i in row:
            val = i[0]
            if val != '...':
                lst.append(val)
        return lst
    else:
        return row     

In [16]:
# Отделяем значения характеристик от второстепенных данных списка

for i in merg:
    df1[i[0]] = df1[i[0]].apply(sort)

In [17]:
# Определяем функции по выделению имен, ссылок актеров и оценок кинокритиков

def name_actor(row):
    n = []
    if row[2]:        
        dct = row[2]
        for i in dct:
            name = i['actor_name']
            n.append(name)
    else:
        n = n
    return n

def link_actor(row):
    n = []
    if row[2]:        
        dct = row[2]
        for i in dct:
            link = i['link']
            n.append(link)
    else:
        n = n
    return n

def rate_critic(row):    
    if row[3]:        
        rate = row[3][0]        
    else:
        rate = 'none'
    return rate

def grade_critic(row):    
    if row[3]:        
        grade = row[3][1]        
    else:
        grade = 'none'
    return grade

In [18]:
# Разбиваем актеров и рейтинги критиков по отдельным столбцам

df1['Актеры'] = df1['charlist'].apply(name_actor)
df1['Актеры ссылки'] = df1['charlist'].apply(link_actor)
df1['Рейтинг критиков'] = df1['charlist'].apply(rate_critic)
df1['Оценок критиков'] = df1['charlist'].apply(grade_critic)
df1[['Оценок критиков', 'Баллы критиков']] = df1['Оценок критиков'].str.split(' оцен', 1, expand= True)

In [29]:
df2 = df1[['name', 'type', 'charlist', 'Полное название',
       'Рейтинг', 'Оценок', 'Полное описание', 'Год производства', 'Платформа',
       'Страна', 'Жанр', 'Слоган', 'Режиссер', 'Сценарий', 'Продюсер',
       'Оператор', 'Композитор', 'Художник', 'Монтаж', 'Премьера в Росcии',
       'Премьера в мире', 'Цифровой релиз', 
       'Возраст', 'Время', 'Бюджет', 'Сборы в США', 'Сборы в мире',
       'Сборы в России', 'Рейтинг MPAA', 'Актеры', 'Актеры ссылки',
       'Рейтинг критиков', 'Оценок критиков', 'Баллы критиков', 'links']].copy()

In [30]:
df2['Оценок'] = df2['Оценок'].apply(lambda x: int(x[:x.find('оцен')].replace(' ', '')) if x and x != 'none' else np.nan)
df2['Рейтинг критиков'] = df2['Рейтинг критиков'].apply(lambda x: int(x[:-1]) if x != '–' and x != 'none' else np.nan)
df2['Оценок критиков'] = df2['Оценок критиков'].apply(lambda x: int(x) if x != 'none' else np.nan)
df2['Баллы критиков'] = df2['Баллы критиков'].apply(lambda x: float(x[2:]) if x and len(x)>2 else np.nan)

In [32]:
df2['Рейтинг'] = df2['Рейтинг'].apply(lambda x: np.nan if '%' in x or x == 'none' or x == '–' else x)
df2['Рейтинг'] = df2['Рейтинг'].apply(lambda x: float(x) if x else np.nan)

df2['Кол-во сезонов'] = df2['Год производства'].apply(lambda x: int(x[1][:x[1].find('сез') - 1]) if x != '—' and len(x) == 2 else np.nan)
df2['Год производства'] = df2['Год производства'].apply(lambda x: int(x[0]) if x != '—' and x else np.nan)

df2['Жанр'] = df2['Жанр'].apply(lambda x: x[:-1] if 'слова' in x else x)

df2['Прокатчик'] = df2['Премьера в Росcии'].apply(lambda x: x[1] if len(x) == 2 else 'none')
df2['Премьера в Росcии'] = df2['Премьера в Росcии'].apply(lambda x: x[0] if x and x != 'none' else 'none')

df2['Премьера в мире'] = df2['Премьера в мире'].apply(lambda x: x[0] if x != 'none' else x)

df2['Релизер'] = df2['Цифровой релиз'].apply(lambda x: x[1] if x!='none' and len(x)>1 else x)
df2['Цифровой релиз'] = df2['Цифровой релиз'].apply(lambda x: x[0] if x != 'none' else 'none')

df2['Возраст'] = df2['Возраст'].apply(lambda x: int(x[:-1]) if x != 'none' else np.nan)

df2['Время'] = df2['Время'].apply(lambda x: int(x[:x.find('мин') - 1]) if x != '—' else np.nan)

In [33]:
def world_check(row):
    if row != 'none':
        sbor = row[0]
        if sbor[0] == '+':
            n = sbor.find('=')
            check = sbor[n+2:]
        else:
            check = sbor
        check = check.replace(u'\xa0', u' ').replace(' ', '')
    else:
        check = np.nan
    return check

In [34]:
df2['Бюджет'] = df2['Бюджет'].apply(lambda x: x[0].replace(u'\xa0', u' ').replace(' ', '') if x != 'none' else np.nan)
df2['Сборы в США'] = df2['Сборы в США'].apply(lambda x: x[0].replace(u'\xa0', u' ').replace(' ', '') if x != 'none' else np.nan)
df2['Сборы в России'] = df2['Сборы в России'].apply(lambda x: x[0].replace(u'\xa0', u' ').replace(' ', '') if x != 'none' else np.nan)
df2['Сборы в мире'] = df2['Сборы в мире'].apply(world_check)

In [35]:
df2['Рейтинг MPAA'] = df2['Рейтинг MPAA'].apply(lambda x: x.replace('NC-17', '') \
                                              .replace('PG-13', '') \
                                              .replace('PG', '') \
                                              .replace('R', '') \
                                              .replace('G', '') if x != 'none' else x)

In [36]:
df2[['Оценок', 'Год производства', 'Возраст', 'Время', 'Рейтинг критиков', 'Оценок критиков', 'Кол-во сезонов']] = df2[['Оценок', 'Год производства', 'Возраст', 'Время', 'Рейтинг критиков', 'Оценок критиков', 'Кол-во сезонов']].astype('Int64')

df2.rename(columns={'Возраст': 'Возраст, лет+', 'Время': 'Время, мин', 'Рейтинг критиков': 'Рейтинг критиков, %'}, inplace=True)

In [37]:
month_num = {'января': '01', 
             'февраля': '02', 
             'марта': '03', 
             'апреля': '04', 
             'мая': '05', 
             'июня': '06', 
             'июля': '07', 
             'августа': '08', 
             'сентября': '09', 
             'октября': '10', 
             'ноября': '11', 
             'декабря': '12', 
             'январь': '01', 
             'февраль': '02', 
             'март': '03', 
             'апрель': '04', 
             'май': '05', 
             'июнь': '06', 
             'июль': '07', 
             'август': '08', 
             'сентябрь': '09', 
             'октябрь': '10', 
             'ноябрь': '11', 
             'декабрь': '12'}

def month_transform(row):
    if row != 'none':
        lst = row.split(' ')
        md = month_num[lst[1]]
        lst[1] = md
        x = ' '.join(lst)
    else:
        x = np.nan
    return x

# def date_ru(row):
#     if row != 'none':
#         if ' ' in row:           
#             lst = row.split(' ')
#             if len(lst) == 3:
#                 md = month_num[lst[1]]
#                 lst[1] = md
#                 x = ' '.join(lst)
#             elif len(lst) == 2:
#                 md = month_num[lst[0]]
#                 lst[0] = md
#                 x = ' '.join(lst)
#         else:
#             x = row
#     else:
#         x = np.nan
#     return x 


def date_ru(row):
    if row != 'none':
        if ' ' in row:           
            lst = row.split(' ')
            md = month_num[lst[1]]
            lst[1] = md
            x = ' '.join(lst)
        else:
            x = np.nan
    else:
        x = np.nan
    return x 

def datetime(row):
    if row:
        t = dt.datetime.strptime(row, '%d %m %Y')
    else:
        t = row
    print(t.date())
    

In [330]:
# df2['Премьера в Росcии'] = df2['Премьера в Росcии'].apply(date_ru)
# df2['Премьера в мире'] = df2['Премьера в мире'].apply(month_transform)
# df2['Цифровой релиз'] = df2['Цифровой релиз'].apply(month_transform)

df2['Премьера в Росcии'] = df2['Премьера в Росcии'].apply(datetime)
# df2['Премьера в мире'] = df2['Премьера в мире'].apply(datetime)
# df2['Цифровой релиз'] = df2['Цифровой релиз'].apply(datetime)

2011-09-08
2014-09-08
2015-10-01
2015-10-08
2015-06-18
2009-01-07
2013-01-04
2016-01-14
2013-06-18
2015-02-12
2010-11-04
2007-02-18
2010-04-10
2015-04-23
2016-01-07
2015-05-14
2015-09-24
2014-01-13
2012-10-22
2015-05-21
2015-07-16
2010-12-13
2013-07-01
2016-06-30
2015-10-18
2016-01-21
2015-07-09
2015-12-17
2015-10-29
2000-03-04
2015-12-19
2015-06-11
2016-10-10
2015-06-04
2000-06-21
2015-08-13
2010-08-16
2015-04-09
2015-02-12
2015-03-05
2015-10-24
2013-06-06
2017-07-13
2015-04-02
2010-03-29
2015-03-26
2009-11-02
2017-02-02
2006-09-01
2014-08-19
2015-12-24
2013-11-11
2016-09-05
2015-03-19
2012-09-20
2015-11-19
2015-09-24
2015-10-15
2015-07-30
2015-09-17
2015-07-02
2015-12-03
2015-07-23


TypeError: strptime() argument 1 must be str, not float

In [38]:
bgt = df2['Премьера в Росcии'][123470]
bgt

'3 мая 2018'

In [325]:
datetime(bgt)

2018-05-03


In [318]:
date_obj = dt.datetime.strptime(bgt, '%d %m %Y')
print(type(date_obj.date()))

<class 'datetime.date'>


In [329]:
df2['Премьера в Росcии'][df2['Премьера в Росcии'] != 'none'].value_counts()

27 09 2018    22
9 12 2021     22
12 09 2019    21
22 11 2018    21
15 10 2020    20
              ..
27 03 2016     1
12 04 2013     1
31 05 2015     1
7 07 2011      1
16 11 2023     1
Name: Премьера в Росcии, Length: 2883, dtype: int64

In [332]:
list(df2['Премьера в Росcии'].value_counts().index)

['27 09 2018',
 '9 12 2021',
 '12 09 2019',
 '22 11 2018',
 '15 10 2020',
 '24 11 2022',
 '17 11 2022',
 '2 12 2021',
 '1 12 2016',
 '21 11 2019',
 '18 10 2018',
 '8 12 2022',
 '8 03 2018',
 '5 12 2019',
 '7 11 2019',
 '22 09 2016',
 '14 03 2019',
 '13 08 2020',
 '19 09 2019',
 '27 02 2020',
 '15 09 2022',
 '15 03 2018',
 '20 09 2018',
 '20 10 2022',
 '14 04 2018',
 '14 04 2022',
 '26 09 2019',
 '3 11 2022',
 '27 10 2022',
 '1 07 2021',
 '10 11 2022',
 '29 09 2022',
 '23 09 2021',
 '1 10 2020',
 '29 08 2019',
 '26 10 2017',
 '11 03 2021',
 '29 10 2020',
 '8 11 2018',
 '23 11 2017',
 '1 04 2021',
 '21 04 2018',
 '24 10 2019',
 '10 12 2020',
 '28 02 2019',
 '6 02 2020',
 '21 10 2021',
 '27 10 2016',
 '10 06 2021',
 '20 04 2018',
 '14 11 2019',
 '3 12 2020',
 '16 09 2021',
 '14 02 2019',
 '6 09 2018',
 '14 10 2021',
 '11 04 2019',
 '20 08 2020',
 '24 06 2021',
 '15 04 2021',
 '26 11 2020',
 '1 04 2019',
 '11 10 2018',
 '27 04 2017',
 '27 05 2021',
 '5 10 2020',
 '12 11 2015',
 '11 02 2021

In [278]:
list(df2['Премьера в Росcии'].values)

['8 сентября 2011',
 '8 сентября 2014',
 '1 октября 2015',
 '8 октября 2015',
 '18 июня 2015',
 '7 января 2009',
 '4 января 2013',
 '14 января 2016',
 '18 июня 2013',
 '12 февраля 2015',
 '4 ноября 2010',
 '18 февраля 2007',
 '10 апреля 2010',
 '23 апреля 2015',
 '7 января 2016',
 '14 мая 2015',
 '24 сентября 2015',
 '13 января 2014',
 '22 октября 2012',
 '21 мая 2015',
 '16 июля 2015',
 '13 декабря 2010',
 '1 июля 2013',
 '30 июня 2016',
 '18 октября 2015',
 '21 января 2016',
 '9 июля 2015',
 '17 декабря 2015',
 '29 октября 2015',
 '4 марта 2000',
 '19 декабря 2015',
 '11 июня 2015',
 '10 октября 2016',
 '4 июня 2015',
 '21 июня 2000',
 '13 августа 2015',
 '16 августа 2010',
 '9 апреля 2015',
 '12 февраля 2015',
 '5 марта 2015',
 '24 октября 2015',
 '6 июня 2013',
 '13 июля 2017',
 '2 апреля 2015',
 '29 марта 2010',
 '26 марта 2015',
 '2 ноября 2009',
 '2 февраля 2017',
 '1 сентября 2006',
 '19 августа 2014',
 '24 декабря 2015',
 '11 ноября 2013',
 '5 сентября 2016',
 '19 марта 2015',

In [251]:
df2[df2['links'] == 'https://www.kinopoisk.ru/film/1143242/']

,name,type,charlist,Полное название,Рейтинг,Оценок,Полное описание,Год производства,Платформа,Страна,...,Рейтинг MPAA,Актеры,Актеры ссылки,"Рейтинг критиков, %",Оценок критиков,Баллы критиков,links,Кол-во сезонов,Прокатчик,Релизер
178550,Джентльмены,film,"[[{'name': 'Джентльмены (2019)', 'description'...",Джентльмены (2019),8.5,1292511,"{'name': 'Джентльмены (2019)', 'description': ...",2019,none,"[Великобритания, США]",...,лицам до 17 лет обязательно присутствие взрослого,"[Мэттью МакКонахи, Чарли Ханнэм, Генри Голдинг...","[/name/797/, /name/38702/, /name/4950097/, /na...",75,275,6.5,https://www.kinopoisk.ru/film/1143242/,<NA>,«Вольга»,«Вольга»


In [327]:
df2.loc[178550]

name                                                         Джентльмены
type                                                                film
charlist               [[{'name': 'Джентльмены (2019)', 'description'...
Полное название                                       Джентльмены (2019)
Рейтинг                                                              8.5
Оценок                                                           1292511
Полное описание        {'name': 'Джентльмены (2019)', 'description': ...
Год производства                                                    2019
Платформа                                                           none
Страна                                             [Великобритания, США]
Жанр                                  [криминал, комедия, боевик, слова]
Слоган                                                 «Criminal. Class»
Режиссер                                                      [Гай Ричи]
Сценарий                          [Гай Ричи, Айван 